In [1]:
import numpy as np
import os

# Background

The overlap matrix is:

$$S_{\mu \nu} =  \int d\vec{r}_{1} \phi_{\mu}(1)^{*}\phi_{\nu}(1)$$

- $\phi_{\mu}$ are basis functions (defined in basis set)


The unknown molecular orbitals $\psi_{i}$ are expanded as a linear expansion of the $K$ known basis functions $\{ \phi_{i} | i=1,2,..., K \}$:

$$ \psi_{i} =  \sum_{\mu=1}^{K} C_{\mu i} \phi_{\mu}$$


$C$ is a $K \times K$ matrix of expansion coefficients $C_{\mu i}$. The columns of $C$ describe the molecular orbitals!


We can find the total number of electrons $N$ in the system by:

$$ N =  2 \sum_{a}^{N/2}\int d\vec{r}  \bigg( \psi_{a}(\vec{r})^{*} \psi_{i}(\vec{r}) \bigg) =  2 \sum_{a}^{N/2} 1$$

- integral gives probablity of finding electron $a$ over all space (must be 1)
- summing over all electrons will give the total number of electrons

The charge density has the following definition:

$$\rho(\vec{r}) = 2 \sum_{a}^{N/2} \bigg( \psi_{a}(\vec{r})^{*} \psi_{i}(\vec{r}) \bigg)$$

- re-write using definition of $\psi_{i}=  \sum_{\mu=1}^{K} C_{\mu i} \phi_{\mu}$

$$\rho(\vec{r}) = 2 \sum_{a}^{N/2} \Bigg( \bigg[ \sum_{\nu}^{K} C_{\nu a}^{*} \phi_{\nu}(\vec{r})^{*} \bigg] \bigg[ \sum_{\mu}^{K} C_{\mu a}\phi_{\mu}(\vec{r}) \bigg] \Bigg)$$

- move things around

$$\rho(\vec{r}) = \sum_{\nu}^{K} \sum_{\mu}^{K} \Big( 2 \sum_{a}^{N/2} C_{\mu a} C_{\nu a}^{*} \Big) \phi_{\mu}(\vec{r}) \phi_{\nu}(\vec{r})^{*} $$

- which is 

$$\rho(\vec{r}) = \sum_{\mu, \nu}^{K} P_{\mu \nu} \phi_{\mu}(\vec{r}) \phi_{\nu}(\vec{r})^{*} $$


- $P_{\mu \nu}$ is known as the density matrix and is:

$$P_{\mu \nu} = 2 \sum_{a}^{N/2} C_{\mu a} C_{\nu a}^{*}$$

Therefore we can also find the total number of electrons in the system by:

$$ N =  2 \sum_{a}^{N/2}\int d\vec{r}  \bigg( \psi_{a}(\vec{r})^{*} \psi_{i}(\vec{r}) \bigg) =  \sum_{\nu}^{K} \sum_{\mu}^{K} \Big( 2 \sum_{a}^{N/2} C_{\mu a} C_{\nu a}^{*} \Big) \int d\vec{r} \phi_{\mu}(\vec{r})  \phi_{\nu}(\vec{r})^{*}$$

- This is simply:

$$N =  \sum_{\nu}^{K} \sum_{\mu}^{K} P_{\mu \nu} S_{\nu \mu}= \sum_{\mu}^{K} PS_{\mu \mu} = \mathcal{Tr}(PS)$$

- One can interpret $ PS_{\mu \mu}$ in the above equation as the number of electrons associated with $ \phi_{\mu}$
- This is a **Mulliken population analysis**

# Orbital Localization

- When we perform a SCF calculation, one gets an optimized C matrix
    - $C$ is a $K \times K$ matrix of expansion coefficients $C_{\mu i}$
    - The columns of $C$ describe the molecular orbitals!
    - MO i: $ \psi_{i} =  \sum_{\mu=1}^{K} C_{\mu i} \phi_{\mu}$
    
    
- These molecular orbitals are usually **delocalized**
    - non-negligible amplitude over the whole system, rather than only around some atom(s) or bond(s)

- But we know in QM that a given basis choice is NOT unique


- We can therefore perform a unitary rotation on molecular orbitals

$$ \psi_{i} U_{rot} =  \Big( \sum_{\mu=1}^{K} C_{\mu i} \phi_{\mu} \Big) U_{rot} = \psi_{i}^{new}$$
    
    
The idea is to use a rotation such that the resulting orbitals $\psi_{i}^{new}$ are as spatially localized as possible. 


The Pipek-Mezey (PM) [localization](https://notendur.hi.is/hj/papers/paperPipekmezey8.pdf) **maximizes the population charges on the atoms**:

$$ f (U_{rot}) = \sum_{A}^{N_{atoms}} \Bigg( Z_{A} -  \sum_{\mu \text{ on atom } A} PS_{\mu \mu} \Bigg)$$

### Method 1
- Given optimized $C$ coefficient matrix
    - which has been rotated to localize orbitals
    - (used to build localized density matrix)


- **Look through basis functions $\phi_{\mu}$ of the ACTIVE atoms**

    
- check the mulliken charge // mulliken population of the orbital
    - if above a certain threshold associate it to active system
    - otherwise put in the environment
 


To choose the active and enviroment subsystems we do the following:

1. Given a localized molecular orbs (localized C matrix), we take the absolute mag squared of the coefficients of the active part for a given localized orb and divide by the absolute mag squared of all the coefficents of a that orb... THis will give a value of how much the active system contributes to that orb.

2. Mathematically, for orbital $j$ 
    - remember MO orbs given by columns of C matrix
    - In equation below C matrix is the LOCALIZED form!


$$ \text{threshold} =  \frac{\sum_{\mu\in \text{active AO}}^{K} |C_{\mu j}|^{2}}{\sum_{\mu =1}^{K} |C_{\mu j}|^{2}}$$

## METHOD 2 - SPADE

see https://pubs.acs.org/doi/10.1021/acs.jctc.8b01112 for further info

In [2]:
# get xyz file for water

notebook_dir = os.getcwd()
docs_dir = os.path.dirname(notebook_dir)
NBed_dir = os.path.dirname(docs_dir)
Test_dir = os.path.join(NBed_dir, "tests")
mol_dir = os.path.join(Test_dir, "molecules")

water_xyz_path = os.path.join(mol_dir, "water.xyz")

In [3]:
### inputs
from pyscf import gto, scf

basis = "STO-3G"
charge = 0
spin = 0
full_system_mol = gto.Mole(
    atom=water_xyz_path,
    basis=basis,
    charge=charge,
    spin=spin,
)
full_system_mol.build()

HF_scf = scf.RHF(full_system_mol)
HF_scf.verbose = 1
HF_scf.conv_tol = 1e-6
HF_scf.kernel()
###

-74.96099960300506

In [4]:
from nbed.localizers import (
    BOYSLocalizer,
    IBOLocalizer,
    Localizer,
    PMLocalizer,
    SPADELocalizer,
)

/home/mwilliams/.cache/pypoetry/virtualenvs/nbed-g8dV7HW4-py3.8/lib/python3.8/site-packages/pyscf/dft/libxc.py:772: UserWarning: Since PySCF-2.3, B3LYP (and B3P86) are changed to the VWN-RPA variant, the same to the B3LYP functional in Gaussian and ORCA (issue 1480). To restore the VWN5 definition, you can put the setting "B3LYP_WITH_VWN5 = True" in pyscf_conf.py
  warnings.warn('Since PySCF-2.3, B3LYP (and B3P86) are changed to the VWN-RPA variant, '
/home/mwilliams/.cache/pypoetry/virtualenvs/nbed-g8dV7HW4-py3.8/lib/python3.8/site-packages/_distutils_hack/__init__.py:33: UserWarning: Setuptools is replacing distutils.
  warnings.warn("Setuptools is replacing distutils.")


In [5]:
localizers = {
    "spade": SPADELocalizer,
    "boys": BOYSLocalizer,
    "ibo": IBOLocalizer,
    "pipek-mezey": PMLocalizer,
}

In [6]:
n_active_atoms = 2  # (first n rows are active in xyz file)
loc_str = "boys"  # <--- change to perform different localization
threshold = 0.95


## object runs localization when initialized!
loc_system = localizers[loc_str](
    HF_scf,
    n_active_atoms,
    occ_cutoff=threshold,
    # virt_cutoff=0.95,
    # run_virtual_localization=False
)

In [7]:
print(f"active MO inds: {loc_system.active_MO_inds}")
print(f"enviro MO inds: {loc_system.enviro_MO_inds}")


# orb threshold
if loc_str != "spade":
    print("Localized MO threshold:", loc_system.enviro_selection_condition)
    print(f"indices above {threshold} set active")
    # indices above threshold (usually 95%) set to active

active MO inds: [0 3 4]
enviro MO inds: [1 2]
Localized MO threshold: [0.99988767 0.86614158 0.74106956 0.98629906 1.        ]
indices above 0.95 set active


In [9]:
dm_localised_full_system = 2 * loc_system._c_loc_occ @ loc_system._c_loc_occ.conj().T
dm_active = loc_system.dm_active
dm_enviro = loc_system.dm_enviro

# check act and env density give the full density
print(
    f"does: y_full = y_act + y_env ... {np.allclose(dm_localised_full_system, dm_active + dm_enviro)}"
)


# check number of electrons is still the same after orbitals have been localized (change of basis)
s_ovlp = loc_system._global_ks.get_ovlp()
n_active_electrons = np.trace(dm_active @ s_ovlp)
n_enviro_electrons = np.trace(dm_enviro @ s_ovlp)


# check number of electrons is correct
print(
    f"does: n_elec_full = n_elec_act + n_elec_env ... {np.allclose(HF_scf.mol.nelectron, n_active_electrons+n_enviro_electrons)}"
)

does: y_full = y_act + y_env ... True
does: n_elec_full = n_elec_act + n_elec_env ... True


In [10]:
# active MOs
loc_system.c_active

array([[ 1.01008671e+00, -1.59054590e-01,  4.26034555e-15],
       [-4.40640993e-02,  9.19996115e-01, -3.13826758e-16],
       [-4.16468573e-15, -2.40346266e-16,  1.00000000e+00],
       [ 9.15040240e-14, -4.37684418e-13,  3.35362240e-16],
       [-1.37893210e-02,  5.57339367e-01,  2.95082195e-16],
       [-1.07176792e-02, -1.29055262e-01, -2.75257096e-16],
       [-1.07176792e-02, -1.29055262e-01, -2.74736379e-16]])

In [11]:
# enviro MOs
loc_system.c_enviro

array([[ 8.56924393e-02, -2.56089624e-13],
       [-3.60493326e-01,  6.23136670e-13],
       [-5.32230746e-16, -9.18734758e-17],
       [-8.16432016e-14,  6.05580236e-01],
       [ 5.63722485e-01,  4.81741489e-13],
       [-2.88421742e-01,  4.43789533e-01],
       [-2.88421742e-01, -4.43789533e-01]])

In [12]:
# full localized C matrix
loc_system.

AttributeError: 'BOYSLocalizer' object has no attribute 'c_loc_occ_and_virt'

In [13]:
# to get active and environment C matrix slice this array using active_MO_inds and enviro_MO_inds

## e.g. for environment
np.allclose(
    loc_system.c_loc_occ_and_virt[:, loc_system.enviro_MO_inds], loc_system.c_enviro
)

AttributeError: 'BOYSLocalizer' object has no attribute 'c_loc_occ_and_virt'

See notebook 1 on how to plot these LOCALIZED orbitals

use c_loc_occ_and_virt / c_active / c_enviroment to plot localized orbitals